# Task 3 - Model Explainability

In [6]:
# %pip install shap
# %pip install xgboost

In [7]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import shap
from sklearn.metrics import f1_score, confusion_matrix, average_precision_score
import os
import sys
import ipaddress
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
sys.path.append(os.path.abspath("../Script"))
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, average_precision_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings("ignore")

In [8]:
os.chdir("..")
# print(os.getcwd()) 

In [9]:
from data_load import *
from model_explain import *

In [12]:
    # Load datasets
df = load_and_clean_data("Data/Fraud_Data.csv")
ip_df = load_and_clean_data("Data/IpAddress_to_Country.csv")
credit_df = load_and_clean_data("Data/creditcard.csv")

Loaded: Data/Fraud_Data.csv | Rows: 151112, Columns: 11
Cleaned: Removed duplicates and empty rows.
Loaded: Data/IpAddress_to_Country.csv | Rows: 138846, Columns: 3
Cleaned: Removed duplicates and empty rows.
Loaded: Data/creditcard.csv | Rows: 284807, Columns: 31
Cleaned: Removed duplicates and empty rows.


In [ ]:
X_train_bal_scaled (numpy or pd.DataFrame) after SMOTE + scaling
xgb_model (trained XGBoostClassifier)

# Convert back to DataFrame if needed (important for feature names in SHAP)
if not isinstance(X_train_bal_scaled, pd.DataFrame):
    X_train_df = pd.DataFrame(X_train_bal_scaled, columns=X.columns)
else:
    X_train_df = X_train_bal_scaled

# Interpret model
explain_model_with_shap(model=xgb_model, X_train=X_train_df, X_sample=0)
Call SHAP interpretation function
explain_model_with_shap(best_model, X_train_scaled, X_sample=5)

In [ ]:
# Assuming df is creditcard dataset and already preprocessed
X_train, X_test, y_train, y_test = prepare_data(df, target_col='class')

# Optional: scaling + SMOTE
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train_scaled, y_train)

# Train logistic regression
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train_bal, y_train_bal)

# Train XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train_bal, y_train_bal)

# Evaluate
f1_log, auc_log = evaluate_model(log_model, X_test_scaled, y_test, name="Logistic Regression")
f1_xgb, auc_xgb = evaluate_model(xgb_model, X_test_scaled, y_test, name="XGBoost")

# Choose best model
best_model = xgb_model if f1_xgb > f1_log else log_model
print("\nBest model selected:", "XGBoost" if best_model == xgb_model else "Logistic Regression")

# SHAP explanation
X_train_df = pd.DataFrame(X_train_bal, columns=X_train.columns)
explain_model_with_shap(best_model, X_train_df, X_sample=5)